In [1]:
import pandas as pd

In [2]:
test_df = pd.read_csv('test.csv')

In [3]:
test_df.head()

,Unnamed: 0.1,Unnamed: 0,conference,year,paper_id,title,abstract,topics,image_url,local_image_path,local_pdf_path,arxiv_id,error
0,0,0,ICML,2024,37030,Regularized Distribution Matching Distillation...,Diffusion distillation methods aim to compress...,['Computer Vision' 'Image-to-Image Translation...,https://icml.cc/media/PosterPDFs/ICML%202024/3...,images/37030.png,papers/2406.14762v1_Regularized Distribution M...,2406.14762v1,NaN
1,1,1,NeurIPS,2023,70281,Doubly Robust Augmented Transfer for Meta-Rein...,"Meta-reinforcement learning (Meta-RL), though ...",['Meta-Reinforcement Learning' 'Sparse Reward ...,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,images/70281.png,NaN,NaN,not found
2,2,2,NeurIPS,2022,55264,Learning Best Combination for Efficient N:M Sp...,By forcing N out of M consecutive weights to b...,['Neural Networks' 'Model Compression' 'Sparse...,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,images/55264.png,papers/2206.06662v2_Learning Best Combination ...,2206.06662v2,NaN
3,3,3,NeurIPS,2023,70660,PDF: Point Diffusion Implicit Function for Lar...,Recent advances in implicit neural representat...,['Computer Vision' 'Neural Rendering' '3D Scen...,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,images/70660.png,papers/2311.01773v1_PDF Point Diffusion Implic...,2311.01773v1,NaN
4,4,4,ICLR,2023,11289,Budgeted Training for Vision Transformer,The superior performances of Vision Transforme...,['Computer Vision' 'Deep Learning' 'Model Opti...,https://iclr.cc/media/PosterPDFs/ICLR%202023/1...,images/11289.png,NaN,NaN,not found


In [4]:
sample_unit = test_df.iloc[[0]]

In [5]:
sample_unit

,Unnamed: 0.1,Unnamed: 0,conference,year,paper_id,title,abstract,topics,image_url,local_image_path,local_pdf_path,arxiv_id,error
0,0,0,ICML,2024,37030,Regularized Distribution Matching Distillation...,Diffusion distillation methods aim to compress...,['Computer Vision' 'Image-to-Image Translation...,https://icml.cc/media/PosterPDFs/ICML%202024/3...,images/37030.png,papers/2406.14762v1_Regularized Distribution M...,2406.14762v1,NaN


In [8]:
import os

os.path.isfile(sample_unit.loc[0, 'local_pdf_path'])

True

In [14]:
sample_unit.style

,Unnamed: 0.1,Unnamed: 0,conference,year,paper_id,title,abstract,topics,image_url,local_image_path,local_pdf_path,arxiv_id,error
0,0,0,ICML,2024,37030,Regularized Distribution Matching Distillation for One-step Unpaired Image-to-Image Translation,"Diffusion distillation methods aim to compress the diffusion models into efficient one-step generators while trying to preserve quality. Among them, Distribution Matching Distillation (DMD) offers a suitable framework for training general-form one-step generators, applicable beyond unconditional generation. In this work, we introduce its modification, called Regularized Distribution Matching Distillation, applicable to unpaired image-to-image problems. We demonstrate its empirical performance in application to several translation tasks, including 2D examples and I2I between different image datasets, where it performs on par or better than multi-step diffusion baselines.",['Computer Vision' 'Image-to-Image Translation' 'Deep Learning' 'Generative Models' 'Diffusion Models'],https://icml.cc/media/PosterPDFs/ICML%202024/37030.png,images/37030.png,papers/2406.14762v1_Regularized Distribution Matching Distillation for.pdf,2406.14762v1,nan


In [18]:
import numpy as np
from PIL import Image
import cv2
import imagehash
from skimage.metrics import structural_similarity as ssim
from pathlib import Path

def compute_feature_score(img1_path: str, img2_path: str) -> float:
    pil1 = Image.open(img1_path).convert('RGB')
    pil2 = Image.open(img2_path).convert('RGB')
    
    arr1 = np.array(pil1)
    arr2 = np.array(pil2)
    
    phash1 = imagehash.phash(pil1, hash_size=16)
    phash2 = imagehash.phash(pil2, hash_size=16)
    phash_score = max(0, 1 - (phash1 - phash2) / 64)
    
    size = (256, 256)
    ssim_arr1 = np.array(pil1.convert('L').resize(size))
    ssim_arr2 = np.array(pil2.convert('L').resize(size))
    ssim_score, _ = ssim(ssim_arr1, ssim_arr2, full=True)
    
    gray1 = cv2.cvtColor(arr1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(arr2, cv2.COLOR_RGB2GRAY)
    gray1 = cv2.resize(gray1, (512, 512))
    gray2 = cv2.resize(gray2, (512, 512))
    
    feature_ratio = 0.0
    try:
        sift = cv2.SIFT_create(nfeatures=500)
        kp1, des1 = sift.detectAndCompute(gray1, None)
        kp2, des2 = sift.detectAndCompute(gray2, None)
        
        if des1 is not None and des2 is not None and len(des1) >= 2 and len(des2) >= 2:
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            search_params = dict(checks=50)
            flann = cv2.FlannBasedMatcher(index_params, search_params)
            matches = flann.knnMatch(des1, des2, k=2)
            
            good = [m for m, n in matches if len([m,n]) == 2 and m.distance < 0.7 * n.distance]
            feature_ratio = len(good) / min(len(kp1), len(kp2))
    except:
        pass
    
    combined = 0.3 * phash_score + 0.3 * ssim_score + 0.4 * feature_ratio
    return combined, phash_score, ssim_score, feature_ratio

# Find the specific files
folder1 = Path('./poster_output/markdown/figures/')
folder2 = Path('./paper_output/markdown/figures/')

# Find figure_029
fig029 = folder2 / '37030_figure_029.png'

# Find page_007_figure_005
for f in folder1.iterdir():
    if 'page_007_figure_005' in f.name or 'figure_005' in f.name:
        print(f"Found poster fig: {f.name}")
        score, ph, ss, fr = compute_feature_score(str(f), str(fig029))
        print(f"  vs figure_029:")
        print(f"    Combined: {score:.4f}")
        print(f"    pHash: {ph:.4f}")
        print(f"    SSIM: {ss:.4f}")
        print(f"    Features: {fr:.4f}")

Found poster fig: 2406.14762v1_Regularized Distribution Matching Distillation for_page_007_figure_005.png
  vs figure_029:
    Combined: 0.0756
    pHash: 0.0000
    SSIM: 0.0707
    Features: 0.1360


In [19]:
sample_unit_2 = test_df.iloc[[3]]

In [21]:
sample_unit_2.style

,Unnamed: 0.1,Unnamed: 0,conference,year,paper_id,title,abstract,topics,image_url,local_image_path,local_pdf_path,arxiv_id,error
3,3,3,NeurIPS,2023,70660,PDF: Point Diffusion Implicit Function for Large-scale Scene Neural Representation,"Recent advances in implicit neural representations have achieved impressive results by sampling and fusing individual points along sampling rays in the sampling space. However, due to the explosively growing sampling space, finely representing and synthesizing detailed textures remains a challenge for unbounded large-scale outdoor scenes. To alleviate the dilemma of using individual points to perceive the entire colossal space, we explore learning the surface distribution of the scene to provide structural priors and reduce the samplable space and propose a Point Diffusion implicit Function, PDF, for large-scale scene neural representation. The core of our method is a large-scale point cloud super-resolution diffusion module that enhances the sparse point cloud reconstructed from several training images into a dense point cloud as an explicit prior. Then in the rendering stage, only sampling points with prior points within the sampling radius are retained. That is, the sampling space is reduced from the unbounded space to the scene surface. Meanwhile, to fill in the background of the scene that cannot be provided by point clouds, the region sampling based on Mip-NeRF 360 is employed to model the background representation. Expensive experiments have demonstrated the effectiveness of our method for large-scale scene novel view synthesis, which outperforms relevant state-of-the-art baselines.",['Computer Vision' 'Neural Rendering' '3D Scene Reconstruction' 'Implicit Neural Representations' 'Point Cloud Processing'],https://neurips.cc/media/PosterPDFs/NeurIPS%202023/70660.png,images/70660.png,papers/2311.01773v1_PDF Point Diffusion Implicit Function for Large-s.pdf,2311.01773v1,nan


In [1]:
import pandas as pd

df = pd.read_csv('data/train.csv')

In [2]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'conference', 'year', 'paper_id', 'title',
       'abstract', 'topics', 'image_url', 'local_image_path', 'local_pdf_path',
       'arxiv_id', 'error'],
      dtype='object')

In [4]:
sum(df['local_pdf_path'].isna())

2875

In [5]:
df.shape

(10305, 13)